In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import csv
import re
import time 
import pandas as pd 
driver = webdriver.Chrome()
destination_list = ['Lyon, France']#, 'Nice, France','Bordeaux,France','Marseille,France','Lille,France','Strasbourg,France','Nantes,France','Toulouse,France','Amsterdam,Pays-Bas','Genève, Suisse','Barcelona,Espagne','Brussels, Belgique','Vichy, France','Berlin,Allemagne','Franktfurt,Allemagne']
csv_file=open('Blablacar.csv','a',encoding='utf-8')
writer=csv.writer(csv_file)



for dest in destination_list:
    driver.get("https://www.blablacar.fr/")
    from_box = driver.find_element_by_id("search_from_name")
    to_box = driver.find_element_by_id("search_to_name")
    from_box.send_keys('Paris, France')
    to_box.send_keys(dest)
    submit_button = driver.find_element_by_xpath('//button[@type="submit"]')
    submit_button.click()
    #SCROLL_PAUSE_TIME = 3

    # Get scroll height
    last_height = driver.execute_script("return window.scrollY")
   
       
    while True:
        SCROLL_PAUSE_TIME = 3
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, window.scrollY+1000)")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return window.scrollY")
        if (new_height == last_height):
            break
        last_height = new_height
        
    tag_List=driver.find_elements_by_xpath('//li[@itemtype="http://schema.org/Event"]')
    Link2=[]
    d={}
    
    d['destination']=[]
    d['departure_time'] = []
    d['arrival_time'] = []
    d['price']=[]
    d['picture']=[]
    d['n_views']=[]
    d['n_places']=[]
    d['driver_age']=[]
    d['driver_name']=[]
    d['n_seats']=[]
    d['Trip_Review']=[]
    d['n_reviews']=[]
    d['conduite']=[]
    d['smoking']=[]
    d['pets']=[]
    d['n_annoucements']=[]
    d['ID_verified']=[]
    d['telephone']=[]
    d['n_facebook']=[]
    d['telephone']=[]
    d['email']=[]
    d['gender']=[]
    
    
    
    
    for item in tag_List:
        times = item.find_elements_by_xpath('.//time')
        departure_time=times[0].get_attribute("datetime")
        arrival_time=times[1].get_attribute("datetime")
        d['departure_time'].append(departure_time)
        d['arrival_time'].append(arrival_time)
        d['destination'].append(dest)
        
        Link2.append(item.find_element_by_xpath('.//a').get_attribute("href"))
    
    
    
   
    for url in Link2:
        driver.get(url)
        #scraping for price:
        try:
            price=driver.find_element_by_xpath('.//span[@class="u-right u-textBold u-darkGray size30 padding-right"]').text
        except:
            price="NA"
        d['price'].append(price)
        
        #scraping for picture:
        try:
            picture0=driver.find_element_by_xpath('.//div[@class="ProfileCard-picture"]')
            picture1=picture0.find_element_by_xpath('.//img').get_attribute("src")
            b=picture1.find("avatar")
            if b!=-1:
                picture="picture not available"
            elif b==-1:
                picture="picture available"
        except:
            picture="NA"
        d['picture'].append(picture)
        
        # scraping for number of views and printed date:
        try:
            n_views=driver.find_element_by_xpath('.//span[@class="u-cell"]').text.stip
        except:
            n_views="NA"
        d['n_views'].append(n_views)
        
        # scrap for number of remianing seats
        try:
            remaining_0=driver.find_element_by_xpath(('.//div[@class="Booking-occupancy Block-section"]'))
            remaining=remaining_0.find_element_by_xpath('.//div[@class="padding-top"]')
            remaining_places=remaining.find_element_by_xpath('.//span[@class="u-textBold u-darkGray size20"]').text

        except:
            remaining_places=0
        d['n_places'].append(remaining_places)
        
        # scrap for total number of seats:
        try:
            seats2=driver.find_element_by_xpath('.//ul[@class="u-reset"]')
            seats1=seat.find_elements_by_xpath('.//li[@class="Booking-occupant"]')
            n_seats=len(seats1)
        except:
            n_seats="NA"
        d['n_seats'].append(n_seats)
            

            # scrap for driver age & driver name
        try:
            block_age=driver.find_element_by_xpath('.//div[@class="ProfileCard-infosBlock"]')
            
            age=block_age.find_element_by_xpath('.//div[@class="ProfileCard-info"]').text.strip()
            block_name=driver.find_element_by_xpath('.//div[@class="ProfileCard-infosBlock"]')
            block_name1=block_name.find_element_by_xpath('.//h4[@class="ProfileCard-info ProfileCard-info--name u-truncate"]')
            name=block_name1.find_element_by_xpath('.//a[@rel="nofollow"]').text
        except:
            age="NA"
        d['driver_age'].append(age)
        d['driver_name'].append(name)
        
        # scrap for trip rating (numerical review)
        try:
            general_review=driver.find_element_by_xpath('.//p[@class="ratings-container"]')
            numerical_review=general_review.find_element_by_xpath('.//span[@class="u-textBold u-darkGray"]').text
            
        except:
            numerical_review="NA"
        d['Trip_Review'].append(numerical_review)

            # scrap for trip rating (number of  review)
        try:
            general_review=driver.find_element_by_xpath('.//p[@class="ratings-container"]')
            number_of_reviews=general_review.find_element_by_xpath('.//span[@class="u-gray"]').text
        except:
            number_of_reviews="NA"
        d['n_reviews'].append(number_of_reviews)



            #scrap driving rating 
        try:
            block_conduite=driver.find_element_by_xpath('.//div[@class="Block-section"]')
            profile=block_conduite.find_element_by_xpath('.//div[@class="ProfileCard"]')
            conduite=profile.find_element_by_xpath('.//div[@class="ProfileCard-row"][2]').text.strip()
        except:
            conduite="NA"
        d['conduite'].append(conduite)
        
        #scrap for smoking permition
        try:
            regulations=driver.find_element_by_xpath('.//div[@class="ProfileCard-row u-clearfix"]')
            smoking=regulations.find_element_by_xpath('.//span[@class="no-smoking prefs tip"]').get_attribute("oldtitle")
            if smoking=="La cigarette me dérange.":
                smoking="smoking not allowed"
            if smoking=="La cigarette ne me dérange pas.":
                smoking=="smoking allowed"
        except:  
            smoking="smoking allowed"
        d['smoking'].append(smoking)
        
        
            # scrap for pets acceptable 
        try:
            regulations=driver.find_element_by_xpath('.//div[@class="ProfileCard-row u-clearfix"]')
            pets=regulations.find_element_by_xpath('.//span[@class="no-pet prefs tip"]').get_attribute("oldtitle")
            if pets=="Je n'ai rien contre les animaux.":
                pets="pets allowed"
            if pets=="Je ne veux pas voyager avec un animal.":
                pets="pets not allowed"
        except:

            pets="pets allowed"

        d['pets'].append(pets)
        
        
        
            #scrap for number of annoucements
        try:
            n_annoucements0=driver.find_element_by_xpath('.//div[@class="Block-section"]')
            n_annoucements1=n_annoucements0.find_element_by_xpath('.//ul[@class="main-column-list u-reset"]')
            n_annoucements= n_annoucements1.find_element_by_xpath('.//li').text
        except:
            n_annoucements="NA"
        
        
        d['n_annoucements'].append(n_annoucements)
        
           #scrap identity card
            
        try:
            verification_1=driver.find_element_by_xpath('.//div[@class="ProfileCard-infosBlock"]')
            liscense=verification_1.find_element_by_xpath('.//div[@class="ProfileCard-info u-blue"]').text
            a=liscense.find("Pièce d'identité vérifiée")
            if (a!=-1):
                ID_verified=True
            elif (a==-1):
                 ID_verified=False
        except:
            ID_verified=False
        
        d['ID_verified'].append(ID_verified)
        
        #scrap to check telephone verfication
        try:
            block_telephone=driver.find_element_by_xpath('.//div[@class="Block-section"]')
            element=block_telephone.find_element_by_xpath('//ul[@class="main-column-list verification-list unstyled"]')
            telephone1=element.find_element_by_xpath('//li[1]/span[@class="u-alignMiddle u-green bold tip"]').text.strip()
            if telephone1=="Téléphone vérifié":
                telephone=" telephone available"
            else:
                telephone=" telephone not available"
                
            
        except:
            telephone="telephone not available"
        
        d['telephone'].append(telephone)
        #scrap to check email verification
        try:
            block_email=driver.find_element_by_xpath('.//div[@class="Block-section"]')
            element_1=driver.find_element_by_xpath('.//ul[@class="main-column-list verification-list unstyled"]')
            email_2=element_1.find_element_by_xpath('.//li[2]/span').get_attribute("oldtitle")
            email_3=email_2.find("vérifié")
            if (email_3!=-1):
                email= "email available"
                
        except:
            email="email not available"
        
        d['email'].append(email)

            # scrap number of facebook freinds for drivers
        try:
            block_1=driver.find_element_by_xpath('.//div[@class="Block-section"]')
            block_2=block_1.find_elment_by_xpath('.//ul[class="main-column-list verification-list unstyled"]')
            n_facebook=block_2.find_element_by_xpath('.//li[3]/span[class="u-alignMiddle u-green bold"]').text
        except:
             n_facebook=0
        
        d['n_facebook'].append(n_facebook)
        
        #try:
           # gender=driver.find_element_by_xpath('.//use').get_attribute("href")
           # if gender=="#icon-avatar-driver-m":
              #  gender = "male"
           # if gender=="#icon-avatar-driver-f":
              #  gender= "female"
        #except:
           # gender="not specified"
        
        #d['gender'].append(gender)
        #writer.writerow(d.keys())
        writer.writerow(d.values()) 
        
        
        
        
        
            
        

       
        
        
           
        
        
        
            
        
    print(d)
#writer.writerow(d.keys())
#writer.writerows((zip(*d.values()))) 
#keys = list(d.keys())
#values = list(d.values())
#data = pd.DataFrame({keys[0]:values[0],keys[1]:values[1],keys[2]:values[2],keys[3]:values[3],keys[4]:values[4],keys[5]:values[5],keys[6]:values[6],keys[7]:values[7],keys[8]:values[8],keys[9]:values[9],keys[10]:values[10],keys[11]:values[11],keys[12]:values[12],keys[13]:values[13],keys[14]:values[14],keys[15]:values[15],keys[16]:values[16],keys[17]:values[17],keys[18]:values[18],keys[19]:values[19]})

#data.to_csv('Blablacar.csv',index=False)

{'destination': ['Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, France', 'Lyon, 

{'destination': ['Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France', 'Nice, France'], 'departure_time': ['2018-12-22T05:00:00', '2018-12-22T15:30:00', '2018-12-23T09:00:00', '2018-12-23T10:00:00', '2018-12-23T10:00:00', '2018-12-23T10:00:00', '2018-12-23T10:20:00', '2018-12-23T13:00:00', '2018-12-23T20:00:00', '2018-12-24T08:00:00', '2018-12-24T15:00:00', '2018-12-25T04:00:00', '2018-12-25T08:00:00', '2018-12-25T08:30:00', '

{'destination': ['Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France', 'Bordeaux,France'], 'departure_time': ['2018-12-22T23:00:00', '2018-12-22T05:00:00', '2018-12-22T07:00:00', '2018-12-22T08:00:00', '2018-12-22T08:00:00', '2018-12-22T09:00:00', '2018-12-22T09:50:00', '2018-12-22T10:00:00', '2018-12-22T10:00:00', '2018-12-22T12:40:00', '2018-12-22T13:00:00'], 'arrival_time': ['2018-12-23T04:50:00', '2018-12-22T10:50:00', '2018-12-22T13:00:00', '2018-12-22T13:40:00', '2018-12-22T14:40:00', '2018-12-22T15:00:00', '2018-12-22T16:00:00', '2018-12-22T16:20:00', '2018-12-22T15:30:00', '2018-12-22T19:10:00', '2018-12-22T19:00:00'], 'price': ['47,50 €', '53,50 €', '53,50 €', '41,50 €', '47,50 €', '53,50 €', '65,00 €', '46,50 €', '53,50 €', '65,00 €', '51,50 €'], 'picture': ['picture available', 'picture available', 'picture available', 'picture available', '

{'destination': ['Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France', 'Lille,France'], 'departure_time': ['2018-12-22T20:00:00', '2018-12-22T05:50:00', '2018-12-22T06:00:00', '2018-12-22T06:00:00', '2018-12-22T06:00:00', '2018-12-22T06:30:38', '2018-12-22T07:20:00', '2018-12-22T07:30:00', '2018-12-22T07:30:00', '2018-12-22T08:00:00', '2018-12-22T08:10:00'], 'arrival_time': ['2018-12-22T22:50:00', '2018-12-22T08:10:00', '2018-12-22T08:00:00', '2018-12-22T08:40:00', '2018-12-22T08:40:00', '2018-12-22T09:20:38', '2018-12-22T09:50:00', '2018-12-22T10:00:00', '2018-12-22T09:50:00', '2018-12-22T10:30:00', '2018-12-22T10:30:00'], 'price': ['17,00 €', '15,50 €', '14,50 €', '18,00 €', '18,00 €', '15,50 €', '17,00 €', '15,50 €', '20,50 €', '23,00 €', '18,00 €'], 'picture': ['picture available', 'picture available', 'picture not available', 'picture not available', 'picture not available', '

{'destination': ['Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France', 'Toulouse,France'], 'departure_time': ['2018-12-22T07:50:00', '2018-12-22T20:00:00', '2018-12-22T23:30:00', '2018-12-23T03:10:00', '2018-12-23T05:00:00', '2018-12-23T05:00:00', '2018-12-23T07:00:00', '2018-12-23T08:00:00', '2018-12-23T09:30:00', '2018-12-23T11:00:00'], 'arrival_time': ['2018-12-22T13:50:00', '2018-12-23T03:10:00', '2018-12-23T06:30:00', '2018-12-23T10:20:00', '2018-12-23T11:20:00', '2018-12-23T12:00:00', '2018-12-23T14:10:00', '2018-12-23T15:00:00', '2018-12-23T15:40:00', '2018-12-23T17:20:00'], 'price': ['56,00 €', '61,50 €', '51,50 €', '60,00 €', '47,50 €', '91,00 €', '43,00 €', '52,50 €', '52,50 €', '54,50 €'], 'picture': ['picture available', 'picture not available', 'picture available', 'picture available', 'picture not available', 'picture available', 'picture not available', 'p

{'destination': ['Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne', 'Barcelona,Espagne'], 'departure_time': ['2018-12-22T05:00:00', '2018-12-22T06:00:00', '2018-12-23T09:30:00', '2018-12-25T10:00:00', '2018-12-26T07:00:00', '2018-12-26T09:00:00', '2018-12-26T14:00:00', '2018-12-26T22:00:00', '2018-12-27T13:00:00', '2018-12-27T17:00:00'], 'arrival_time': ['2018-12-22T15:40:00', '2018-12-22T16:00:00', '2018-12-23T20:00:00', '2018-12-25T21:00:00', '2018-12-26T18:00:00', '2018-12-26T20:00:00', '2018-12-27T00:50:00', '2018-12-27T11:40:00', '2018-12-27T23:40:00', '2018-12-28T04:30:00'], 'price': ['79,00 €', '82,50 €', '100,50 €', '82,50 €', '100,50 €', '78,00 €', '82,50 €', '88,50 €', '88,50 €', '94,50 €'], 'picture': ['picture not available', 'picture available', 'picture available', 'picture available', 'picture not available', 'picture available', 'pict

{'destination': ['Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne', 'Berlin,Allemagne'], 'departure_time': ['2018-12-22T21:30:00', '2018-12-23T09:00:00', '2018-12-26T08:30:00', '2018-12-26T13:00:00', '2018-12-28T10:00:00', '2018-12-29T04:00:00', '2018-12-29T10:00:00', '2019-01-02T09:00:00', '2019-01-02T11:00:00', '2019-01-04T07:00:00'], 'arrival_time': ['2018-12-23T09:00:00', '2018-12-23T19:40:00', '2018-12-26T20:30:00', '2018-12-27T00:30:00', '2018-12-28T22:10:00', '2018-12-29T15:20:00', '2018-12-29T20:50:00', '2019-01-02T21:20:00', '2019-01-03T00:40:00', '2019-01-04T19:20:00'], 'price': ['76,50 €', '94,50 €', '84,00 €', '78,00 €', '82,50 €', '79,00 €', '82,50 €', '65,00 €', '80,00 €', '88,50 €'], 'picture': ['picture available', 'picture available', 'picture available', 'picture not available', 'picture available', 'picture available', 'picture available', 

In [2]:
keys = list(d.keys())
values = list(d.values())
data = pd.DataFrame({keys[0]:values[0],keys[1]:values[1],keys[2]:values[2],keys[3]:values[3],keys[4]:values[4],keys[5]:values[5],keys[6]:values[6],keys[7]:values[7],keys[8]:values[8],keys[9]:values[9],keys[10]:values[10],keys[11]:values[11],keys[12]:values[12],keys[13]:values[13],keys[14]:values[14],keys[15]:values[15],keys[16]:values[16],keys[17]:values[17],keys[18]:values[18],keys[19]:values[19]})

data.to_csv('Blablacar.csv',index=False)


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import csv
import re
import time 
import pandas as pd 
driver = webdriver.Chrome()
destination_list = ['Lyon, France']#, 'Nice, France','Bordeaux,France','Marseille,France','Lille,France','Strasbourg,France','Nantes,France','Toulouse,France','Amsterdam,Pays-Bas','Genève, Suisse','Barcelona,Espagne','Brussels, Belgique','Vichy, France','Berlin,Allemagne','Franktfurt,Allemagne']
d={}
d['destination']=[]
d['departure_time'] = []
for dest in destination_list:
    driver.get("https://www.blablacar.fr/")
    from_box = driver.find_element_by_id("search_from_name")
    to_box = driver.find_element_by_id("search_to_name")
    from_box.send_keys('Paris, France')
    to_box.send_keys(dest)
    submit_button = driver.find_element_by_xpath('//button[@type="submit"]')
    submit_button.click()
    
    #SCROLL_PAUSE_TIME = 3
    # Get scroll height
    last_height = driver.execute_script("return window.scrollY")
    while True:
        SCROLL_PAUSE_TIME = 3
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, window.scrollY+1000)")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return window.scrollY")
        if (new_height == last_height):
            break
        last_height = new_height
        
    tag_List=driver.find_elements_by_xpath('//li[@itemtype="http://schema.org/Event"]')
    Link2=[]
    for item in tag_List:
        times = item.find_elements_by_xpath('.//time')
        departure_time=times[0].get_attribute("datetime")
        d['destination']=dest
        d['departure_time'].append(departure_time)

csv_file=open('Blablacar.csv','w',encoding='utf-8')
writer=csv.writer(csv_file)
writer.writerow(d.keys())
writer.writerows((zip(*d.values()))) 
    

IndexError: list index out of range